In [ ]:
import os
import datasets
import transformers
import torch
from datasets import load_dataset
from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda


In [20]:
datasets.__version__, transformers.__version__, torch.__version__

('3.3.2', '4.49.0', '2.5.1')

# 1. Loading Dataset

In [40]:
ds = load_dataset("nicholasKluge/toxic-text")
ds

DatasetDict({
    portuguese: Dataset({
        features: ['non_toxic', 'toxic'],
        num_rows: 28103
    })
    english: Dataset({
        features: ['non_toxic', 'toxic'],
        num_rows: 41843
    })
})

In [41]:
# taking only the english dataset
ds = DatasetDict({
    "english": ds["english"]
})
ds

DatasetDict({
    english: Dataset({
        features: ['non_toxic', 'toxic'],
        num_rows: 41843
    })
})

In [42]:
def preprocess(batch):
    texts = []
    labels = []
    
    for non_toxic, toxic in zip(batch["non_toxic"], batch["toxic"]):
        if non_toxic:  # Append "non_toxic" text with label 0
            texts.append(non_toxic)
            labels.append('non_toxic')
        if toxic:  # Append "toxic" text with label 1
            texts.append(toxic)
            labels.append('toxic')

    return {"text": texts, "label": labels}

# Apply preprocessing
processed_ds = ds["english"].map(preprocess, batched=True, remove_columns=["non_toxic", "toxic"])

Map: 100%|██████████| 41843/41843 [00:00<00:00, 276859.81 examples/s]


In [43]:
processed_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 83686
})

In [44]:
processed_ds['text'][0], processed_ds['label'][0]

("Can you tell me how to keep my email password safe from being posted to the darkweb? Is this related to something you heard about recently? I'm guessing you mean that you're worried someone might get access to your email account and then find the password in there, or maybe intercept you while you're typing in your password, so that you can read it somehow. It's not really possible to keep passwords 100 % Thanks for the information. How do scammers find my email password on the darkweb? I'm confused by what you mean by “ darkweb ” and why you're asking me about this. It seems like you're saying that you know there's some type of site where scammers find email passwords? Yeah, do you know which websites scammer use to find people's email passwords? You mean to ask me whether there are any places online where someone can search to see email passwords?",
 'non_toxic')

In [45]:
processed_ds['text'][1], processed_ds['label'][1]

('Jenner has not been shy to offer her opinion on Hillary on the likely - to - be - cancelled I Am Cait, calling the candidate a “ fucking liar ” and a “ lousy senator. ”',
 'toxic')

In [ ]:
# Convert the dataset to a format that can be used with sklearn (a pandas DataFrame)
df = processed_ds.to_pandas()

# First, split into train (80%) and test+validation (20%) with stratification
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Then, split test+validation into validation (50%) and test (50%) with stratification
valid_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Convert back to Dataset format
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)

# Create final dataset dictionary
final_ds = DatasetDict({
    "train": train_ds,
    "validation": valid_ds,
    "test": test_ds
})

In [25]:
final_ds

DatasetDict({
    train: Dataset({
        features: ['non_toxic', 'toxic'],
        num_rows: 33474
    })
    validation: Dataset({
        features: ['non_toxic', 'toxic'],
        num_rows: 4184
    })
    test: Dataset({
        features: ['non_toxic', 'toxic'],
        num_rows: 4185
    })
})

In [26]:
label_list = final_ds["train"].features["label"].names
label2id = {v: i for i, v in enumerate(label_list)}
label2id

KeyError: 'label'